In [1]:
# Libraries
from classifiers import *
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
# from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

/home/miguel/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/miguel/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
scaler = StandardScaler()
data = pd.read_pickle("./data_no_cathegorical.pkl")

X = data.values[:,:-1]
scaler.fit(X)
X = scaler.transform(X)
Y = data.values[:,-1]

# Split data into train and test sets
seed = 10
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# Labels for this algortihm has to be either 1 or -1
y_train = np.where(y_train < 0.5, -1, 1)
y_test = np.where(y_test < 0.5, -1, 1)

In [10]:
classifier_list = []

In [11]:
# ADABOOST
AdaBoost_classifier = AdaboostClassifier(n_iterations=100)
AdaBoost_classifier.fit(X_train,y_train)
classifier_list.append(AdaBoost_classifier)

# BAGGING
bagging_classifier = Bagging_classifier(type_classifier="SVM")
bagging_classifier.fit(X_train,y_train)
classifier_list.append(bagging_classifier)

# RANDOM FOREST
RandomForest_Classifier = RandomForestClassifier(max_depth=2, random_state=0)
RandomForest_Classifier.fit(X_train, y_train)
classifier_list.append(RandomForest_Classifier)

# XGBOOST
XGBoost_classifier = XGBClassifier(n_estimators=100)
XGBoost_classifier.fit(X_train,y_train)
classifier_list.append(XGBoost_classifier)


In [12]:
# ADACOST
AdaCost_classifier = AdaCost(n_iterations=100)
AdaCost_classifier.fit(X_train,y_train,cost=0.4)
classifier_list.append(AdaCost_classifier)

# ADAMEC
AdaMEC_classifier = AdaMEC(n_iterations=100)
AdaMEC_classifier = AdaMEC_classifier.fit(X_train,y_train)
classifier_list.append(AdaMEC_classifier)

In [16]:
X_train_majority = X_train[y_train < 1] # human (non-bots) => label -1 : are majority
X_train_minority = X_train[y_train > 0]

N = np.shape(y_train)[0]
pi_mi = len(X_train_minority)/np.shape(y_train)[0]

# X_train_majority = X_train_majority.to_numpy()
# X_train_minority = X_train_minority.to_numpy()

# y_test = np.where(y_test < 0.5, -1, 1)

y_train = y_train.flatten()
y_test = y_test.flatten()

In [17]:
def get_undersampled_dataset(Beta):
    X_train_majority_undersampled = X_train_majority[random.sample(range(1, int((1-pi_mi)*N)),int(Beta*(1-pi_mi)*N)), :]
    X_train_under = np.r_[X_train_majority_undersampled,X_train_minority]
    y_train_under = np.r_[[-1]*len(X_train_majority_undersampled), [1]*len(X_train_minority)]
    
    return X_train_under, y_train_under

In [18]:
# ADABOOST
X_train_under, y_train_under = get_undersampled_dataset(0.62)
AdaBoost_classifier_under = AdaboostClassifier(n_iterations=100)
AdaBoost_classifier_under.fit(X_train_under,y_train_under)
classifier_list.append(AdaBoost_classifier_under)
    
# BAGGING
X_train_under, y_train_under = get_undersampled_dataset(0.58)
bagging_classifier_under = Bagging_classifier(type_classifier="SVM")
bagging_classifier_under.fit(X_train_under,y_train_under)
classifier_list.append(bagging_classifier_under)

# RANDOM FOREST
X_train_under, y_train_under = get_undersampled_dataset(0.52)
RandomForest_Classifier_under = RandomForestClassifier(max_depth=2, random_state=0)
RandomForest_Classifier_under.fit(X_train_under, y_train_under)
classifier_list.append(RandomForest_Classifier_under)
    
# XGBOOST
X_train_under, y_train_under = get_undersampled_dataset(0.7)
XGBoost_classifier_under = XGBClassifier(n_estimators=100)
XGBoost_classifier_under.fit(X_train_under,y_train_under)
classifier_list.append(XGBoost_classifier_under)

# ADACOST
X_train_under, y_train_under = get_undersampled_dataset(0.52)
AdaCost_classifier_under = AdaCost(n_iterations=100)
AdaCost_classifier_under.fit(X_train_under,y_train_under,cost=-0.4)
classifier_list.append(AdaCost_classifier_under)
    
# ADAMEC
X_train_under, y_train_under = get_undersampled_dataset(0.65)
AdaMEC_classifier_under = AdaMEC(n_iterations=100)
AdaMEC_classifier_under = AdaMEC_classifier_under.fit(X_train_under,y_train_under)
classifier_list.append(AdaMEC_classifier_under)

In [8]:
X_data_MB_numpy = np.array(np.r_[4,8,22,3,float(9/(3.7*365)),int(3.7*365),1, 1,1, np.zeros(9),1,np.zeros(38),1])

In [4]:
data.columns

Index(['favorites_count', 'followers_count', 'friends_count', 'statuses_count',
       'average_tweets_per_day', 'account_age_days', 'default_profile_True',
       'default_profile_image_True', 'geo_enabled_True', 'lang_af', 'lang_ar',
       'lang_bg', 'lang_ca', 'lang_cs', 'lang_cy', 'lang_da', 'lang_de',
       'lang_el', 'lang_en', 'lang_es', 'lang_et', 'lang_fa', 'lang_fi',
       'lang_fr', 'lang_he', 'lang_hi', 'lang_hr', 'lang_hu', 'lang_id',
       'lang_it', 'lang_ja', 'lang_ko', 'lang_lt', 'lang_lv', 'lang_mk',
       'lang_ne', 'lang_nl', 'lang_no', 'lang_pa', 'lang_pl', 'lang_pt',
       'lang_ro', 'lang_ru', 'lang_sk', 'lang_sl', 'lang_so', 'lang_sq',
       'lang_sv', 'lang_sw', 'lang_th', 'lang_tl', 'lang_tr', 'lang_uk',
       'lang_ur', 'lang_vi', 'lang_zh-cn', 'lang_zh-tw', 'verified_True',
       'account_type_bot'],
      dtype='object')

In [29]:
MB_consensus = []
for classifier in classifier_list: 
    y_pred = classifier.predict(np.reshape(X_data_MB_numpy,(1,58)))
    MB_consensus.append(y_pred)

In [30]:
MB_consensus

[array([-1]),
 array([-1]),
 array([-1]),
 array([-1]),
 array([-1]),
 array([-1]),
 array([-1]),
 array([-1]),
 array([-1]),
 array([-1]),
 array([-1]),
 array([-1])]

In [31]:
np.shape(MB_consensus)

(12, 1)